In [1]:
%matplotlib

Using matplotlib backend: MacOSX


In [4]:
import pywt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mne.io import RawArray
from mne.channels import read_montage
from mne.epochs import concatenate_epochs
from mne import create_info, find_events, Epochs, concatenate_raws, pick_types
from mne.decoding import CSP

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from glob import glob

from scipy.signal import butter, lfilter, convolve, boxcar

In [6]:
def creat_mne_raw_object(fname,read_events=True):
    """Create a mne raw instance from csv file"""
    # Read EEG file
    data = pd.read_csv(fname)
    
    # get chanel names
    ch_names = list(data.columns[1:])
    
    # read EEG standard montage from mne
    montage = read_montage('standard_1005',ch_names)

    ch_type = ['eeg']*len(ch_names)
    data = 1e-3*np.array(data[ch_names]).T
    
    if read_events:
        # events file
        ev_fname = fname.replace('_data','_events')
        # read event file
        events = pd.read_csv(ev_fname)
        events_names = events.columns[1:]
        events_data = np.array(events[events_names]).T
        
        # define channel type, the first is EEG, the last 6 are stimulations
        ch_type.extend(['stim']*6)
        ch_names.extend(events_names)
        # concatenate event file and data
        data = np.concatenate((data,events_data))
        
    # create and populate MNE info structure
    info = create_info(ch_names,sfreq=500.0, ch_types=ch_type, montage=montage)
    info['filename'] = fname
    
    # create raw object 
    raw = RawArray(data,info,verbose=False)
    
    return raw

In [7]:
subjects = range(1,13)
ids_tot = []
pred_tot = []
y_tot = []

# design a butterworth bandpass filter 
freqs = [7, 30]
b,a = butter(5,np.array(freqs)/250.0,btype='bandpass')

# CSP parameters
# Number of spatial filter to use
nfilters = 4

# convolution
# window for smoothing features
nwin = 250

# training subsample
subsample = 10

# submission file
submission_file = 'split_train_test_pywavelets.csv'
cols = ['HandStart','FirstDigitTouch',
        'BothStartLoadPhase','LiftOff',
        'Replace','BothReleased']

In [9]:
subject = 1
epochs_tot = []
y = []

################ READ DATA (FIRST SIX SERIES AS TRAINING DATA) ###########
fnames = []
for i in range(1, 7):
    fnames.append('train/subj%d_series%d_data.csv' % (subject, i))

# read and concatenate all the files
raw = concatenate_raws([creat_mne_raw_object(fname) for fname in fnames])

# pick eeg signal
picks = pick_types(raw.info,eeg=True)

# Filter data for alpha frequency and beta band
# Note that MNE implement a zero phase (filtfilt) filtering not compatible
# with the rule of future data.
# Here we use left filter compatible with this constraint
raw._data[picks] = lfilter(b,a,raw._data[picks])

# determine start time/position of each event
hand_start_events = find_events(raw,stim_channel='HandStart', verbose=False)
first_digit_touch_events = find_events(raw,stim_channel='FirstDigitTouch', verbose=False)
both_start_load_phase_events = find_events(raw,stim_channel='BothStartLoadPhase', verbose=False)
lift_off_events = find_events(raw,stim_channel='LiftOff', verbose=False)
replace_events = find_events(raw,stim_channel='Replace', verbose=False)
both_released_events = find_events(raw,stim_channel='BothReleased', verbose=False)

# epochs signal for 1.0 second after start of event
hand_start_epochs = Epochs(raw, hand_start_events, {'HandStart' : 1}, tmin=0, tmax=1, proj=False, picks=picks, baseline=None, preload=True, add_eeg_ref=False, verbose=False)
first_digit_touch_epochs = Epochs(raw, first_digit_touch_events, {'FirstDigitTouch' : 1}, tmin=0, tmax=1, proj=False, picks=picks, baseline=None, preload=True, add_eeg_ref=False, verbose=False)
both_start_load_phase_epochs = Epochs(raw, both_start_load_phase_events, {'BothStartLoadPhase' : 1}, tmin=0, tmax=1, proj=False, picks=picks, baseline=None, preload=True, add_eeg_ref=False, verbose=False)
lift_off_epochs = Epochs(raw, lift_off_events, {'LiftOff' : 1}, tmin=0, tmax=1, proj=False, picks=picks, baseline=None, preload=True, add_eeg_ref=False, verbose=False)
replace_epochs = Epochs(raw, replace_events, {'Replace' : 1}, tmin=0, tmax=1, proj=False, picks=picks, baseline=None, preload=True, add_eeg_ref=False, verbose=False)
both_released_epochs = Epochs(raw, both_released_events, {'BothReleased' : 1}, tmin=0, tmax=1, proj=False, picks=picks, baseline=None, preload=True, add_eeg_ref=False, verbose=False)

hand_start_tensor = hand_start_epochs.get_data()
first_digit_touch_tensor = first_digit_touch_epochs.get_data()
both_start_load_phase_tensor = both_start_load_phase_epochs.get_data()
lift_off_tensor = lift_off_epochs.get_data()
replace_tensor = replace_epochs.get_data()
both_released_tensor = both_released_epochs.get_data()

print hand_start_tensor.shape
print first_digit_touch_tensor.shape
print both_start_load_phase_tensor.shape
print lift_off_tensor.shape
print replace_tensor.shape
print both_released_tensor.shape

(192, 32, 501)
(192, 32, 501)
(192, 32, 501)
(192, 32, 501)
(191, 32, 501)
(191, 32, 501)


In [10]:
for i in range(32):
    plt.plot(hand_start_tensor[0][i,:])
plt.savefig('hand_start_plot_0.png', bbox_inches='tight')

In [11]:
for i in range(32):
    plt.plot(first_digit_touch_tensor[0][i,:])
plt.savefig('first_digit_touch_plot_0.png', bbox_inches='tight')

In [19]:
total_instances = len(hand_start_tensor) + len(first_digit_touch_tensor) + len(both_start_load_phase_tensor) + len(lift_off_tensor) + len(replace_tensor) + len(both_released_tensor)
total_instances

1150

In [27]:
# Create feature vector by appending wavelet features (NOT COMPLETE)
#feattr = np.empty((total_instances, ))
curr = 0
for m in hand_start_tensor:
    for i in range(32):
        wd = pywt.wavedec(m[i,:], 'db6', level=6)
        wave_feats = map(lambda x: np.dot(x.T, x), wd)
        #feattr[curr] = wave_feats
        curr += 1

In [24]:
wave_feats

[0.84978878525210078,
 0.17711777008117557,
 0.69056067333006921,
 1.1033921615109803,
 0.14139449457167164,
 0.0010081872178661863,
 3.4548664661936582e-05]

In [175]:
map(lambda x: np.dot(x.T, x) ,hand_start_wd1)

[0.84978878525210089,
 0.17711777008117557,
 0.69056067333006921,
 1.1033921615109803,
 0.14139449457167164,
 0.0010081872178661863,
 3.4548664661936582e-05]

In [106]:
a = [np.array([1, 2, 3])]

In [107]:
map(lambda x: np.dot(x.T, x) , a)

[14]

In [119]:
for subject in subjects:
    epochs_tot = []
    y = []
    
    ################ READ DATA (FIRST SIX SERIES AS TRAINING DATA) ###########
    fnames = []
    for i in range(1, 7):
        fnames.append('train/subj%d_series%d_data.csv' % (1, i))
    
    # read and concatenate all the files
    raw = concatenate_raws([creat_mne_raw_object(fname) for fname in fnames])
       
    # pick eeg signal
    picks = pick_types(raw.info,eeg=True)
    
    # Filter data for alpha frequency and beta band
    # Note that MNE implement a zero phase (filtfilt) filtering not compatible
    # with the rule of future data.
    # Here we use left filter compatible with this constraint
    raw._data[picks] = lfilter(b,a,raw._data[picks])
    
    ################ CSP Filters training #####################################
    # get event posision corresponding to Replace
    events = find_events(raw,stim_channel='Replace', verbose=False)
    # epochs signal for 1.5 second before the movement
    epochs = Epochs(raw, events, {'during' : 1}, -2, -0.5, proj=False,
                    picks=picks, baseline=None, preload=True,
                    add_eeg_ref=False, verbose=False)
    
    epochs_tot.append(epochs)
    y.extend([1]*len(epochs))
    
    # epochs signal for 1.5 second after the movement, this correspond to the 
    # rest period.
    epochs_rest = Epochs(raw, events, {'after' : 1}, 0.5, 2, proj=False,
                    picks=picks, baseline=None, preload=True,
                    add_eeg_ref=False, verbose=False)
    
    # Workaround to be able to concatenate epochs with MNE
    epochs_rest.times = epochs.times
    
    y.extend([-1]*len(epochs_rest))
    epochs_tot.append(epochs_rest)
        
    # Concatenate all epochs
    epochs = concatenate_epochs(epochs_tot)
    
    # get data 
    X = epochs.get_data()
    y = np.array(y)
    
    # train CSP
    csp = CSP(n_components=nfilters, reg='lws')
    csp.fit(X,y)
    
    ################ Create Training Features #################################x`
    # apply csp filters and rectify signal
    feat = np.dot(csp.filters_[0:nfilters],raw._data[picks])**2
    
    # smoothing by convolution with a rectangle window    
    feattr = np.empty(feat.shape)
    for i in range(nfilters):
        feattr[i] = np.log(convolve(feat[i],boxcar(nwin),'full'))[0:feat.shape[1]]
    
    # training labels
    # they are stored in the 6 last channels of the MNE raw object
    labels = raw._data[32:]
    
    ################ Create test Features #####################################
    # read test data (remaining series of train data)
    fnames = []
    for i in range(7, 9):
        fnames.append('train/subj%d_series%d_data.csv' % (1, i))
        
    raw = concatenate_raws([creat_mne_raw_object(fname) for fname in fnames])
    raw._data[picks] = lfilter(b,a,raw._data[picks])
    
    # get actual y values
    y = raw._data[32:]
    y_tot.append(y.T)
    
    # read ids
    ids = np.concatenate([np.array(pd.read_csv(fname)['id']) for fname in fnames])
    ids_tot.append(ids)
    
    # apply preprocessing on test data
    feat = np.dot(csp.filters_[0:nfilters],raw._data[picks])**2
    featte = np.empty(feat.shape)
    for i in range(nfilters):
        featte[i] = np.log(convolve(feat[i],boxcar(nwin),'full'))[0:feat.shape[1]]
    
    ################ Train classifiers ########################################
    lr = LogisticRegression()
    pred = np.empty((len(ids),6))
    for i in range(6):
        print('Train subject %d, class %s' % (subject, cols[i]))
        lr.fit(feattr[:,::subsample].T,labels[i,::subsample])
        pred[:,i] = lr.predict_proba(featte.T)[:,1]
    
    pred_tot.append(pred)

Train subject 1, class HandStart
Train subject 1, class FirstDigitTouch
Train subject 1, class BothStartLoadPhase
Train subject 1, class LiftOff
Train subject 1, class Replace
Train subject 1, class BothReleased
Train subject 2, class HandStart
Train subject 2, class FirstDigitTouch
Train subject 2, class BothStartLoadPhase
Train subject 2, class LiftOff
Train subject 2, class Replace
Train subject 2, class BothReleased
Train subject 3, class HandStart
Train subject 3, class FirstDigitTouch
Train subject 3, class BothStartLoadPhase
Train subject 3, class LiftOff
Train subject 3, class Replace
Train subject 3, class BothReleased
Train subject 4, class HandStart
Train subject 4, class FirstDigitTouch
Train subject 4, class BothStartLoadPhase
Train subject 4, class LiftOff
Train subject 4, class Replace
Train subject 4, class BothReleased
Train subject 5, class HandStart
Train subject 5, class FirstDigitTouch
Train subject 5, class BothStartLoadPhase
Train subject 5, class LiftOff
Train s

In [120]:
y_score = np.concatenate(pred_tot)
y_true = np.concatenate(y_tot)

# Convert y_true to int matrix, auc calc will not work with floats
y_true = y_true.astype(int)
auc = roc_auc_score(y_true, y_score, average='macro')
print 'Final AUC:', auc

Final AUC: 0.633816663314
